## 크롤링(웹 크롤링)
크롤링이란
- 인터넷 상의 웹 페이지 데이터를 자동으로 수집하는 과정.
- 웹 크롤링은 일반적으로 웹 스크래핑과 연관되며, 둘은 종종 혼용되지만 조금 다른 개념. 웹 크롤링은 웹 페이지를 탐색하고 데이터를 수집하는 반면, 웹 스크래핑은 그 페이지에서 특정 정보를 추출하는 데 중점을 둔다.
- 크롤링은 스크래핑을 포함할 수 있다. 크롤링 과정에서 각 페이지를 방문할 때, 스크래핑을 통해 필요한 데이터를 추출할 수 있다.

웹 크롤링에 사용되는 도구
- BeautifulSoup: Python 라이브러리로, HTML 및 XML 문서를 구문 분석하고 데이터를 추출하는 데 사용.

- Scrapy: 웹 크롤링을 위한 Python 프레임워크로, 효율적이고 확장성이 높은 크롤러를 쉽게 만들 수 있다.

- Selenium: 웹 브라우저 자동화 도구로, JavaScript가 동적으로 로드되는 페이지를 크롤링할 때 유용.

- Requests: Python의 HTTP 라이브러리로, 웹 페이지 요청을 쉽게 할 수 있다.

웹 크롤링의 기본 과정
- 크롤러 설정: 크롤러는 특정 웹 페이지를 시작점으로 설정. 이를 '시드(seed)'라고 부르며, 크롤러는 이 시드 URL에서 시작해 다른 페이지로 이동.

- 페이지 요청: 크롤러는 HTTP 요청을 보내 웹 페이지를 요청. 이 과정에서 크롤러는 브라우저처럼 행동하여 웹 서버에서 페이지를 가져온다.

- 데이터 추출: 웹 페이지가 응답되면, 크롤러는 페이지의 HTML을 분석하고 필요한 데이터를 추출. 이 과정에는 BeautifulSoup, Selenium 같은 라이브러리가 사용될 수 있다.

- 링크 추출 및 큐잉: 크롤러는 현재 페이지에서 다른 페이지로 연결되는 링크를 추출하고, 이 링크들을 큐(queue)에 추가하여 다음 크롤링 대상으로 삼는다.

- 반복: 이 과정은 정해진 규칙이나 종료 조건이 충족될 때까지 반복. 예를 들어, 특정 수의 페이지를 크롤링하거나, 주어진 도메인 내에서만 크롤링하도록 설정할 수 있다.

웹 크롤링의 주의사항

- 로봇 배제 표준(robots.txt): 많은 웹사이트는 robots.txt 파일을 통해 크롤러가 접근 가능한 부분과 접근을 제한하는 부분을 명시. 크롤러는 이 규칙을 준수해야 한다.

- 저작권 및 법적 이슈: 모든 웹사이트의 콘텐츠는 저작권의 보호를 받는다. 따라서 크롤링을 통해 수집한 데이터를 어떻게 사용할지에 대한 법적 문제를 주의해야 한다.

- 서버 부하: 지나친 크롤링은 웹 서버에 부하를 줄 수 있다. 크롤링 시에는 서버의 부담을 줄이기 위해 요청 간의 딜레이를 설정하는 것이 좋다.

#### 크롤링 사례

In [ ]:

import requests
from bs4 import BeautifulSoup
from collections import deque
import urllib.robotparser
from urllib.parse import urljoin
import time

# robots.txt 파일을 파싱하고 분석하기 위해서는 urllib.robotparser 모듈을 사용
def can_fetch_url(url):
    robot_parser = urllib.robotparser.RobotFileParser()
    robot_parser.set_url('https://example.com/robots.txt')
    robot_parser.read()
    return robot_parser.can_fetch('*', url) # '*'는 모든 사용자 에이전트를 의미. 반환값은 True 또는 False

def get_response_with_retries(url, max_retries=5, backoff_factor=0.3):
    for i in range(max_retries):
        try:
            response = requests.get(url)
            response.raise_for_status()  # HTTP 응답 코드가 200이 아닌 경우 예외 발생
            return response
        except requests.ConnectionError as e: # 네트워크 문제나 서버가 응답하지 않는 경우
            print(f"Connection error: {e}")
        except requests.HTTPError as e: # 클라이언트 오류(4xx) 또는 서버 오류(5xx) 응답 코드
            print(f"HTTP error: {e}")
        except requests.RequestException as e: # 기타 모든 요청 관련 예외(RequestException)를 처리
            print(f"Request failed: {e}")
        time.sleep(backoff_factor * (2 ** i))  # 지수 백오프의 기본 원리는 재시도할 때마다 대기 시간을 두 배로 늘리는 것. 백오프 시간=기본 시간×(2의 n승)
    return None

start_url = 'https://example.com'
# 이미 방문한 URL들을 저장하는 집합
visited = set() # 집합을 사용하여 중복된 URL 방문을 피할 수 있다.
# 방문할 URL들을 저장하는 큐
queue = deque([start_url]) # deque는 양쪽 끝에서 빠르게 추가 및 제거할 수 있는 양방향 큐

max_urls = 10  # 최대 방문할 URL 수
visited_count = 0 # 현재까지 방문한 URL 수

# BFS(Breadth-First Search) 알고리즘을 사용하여 큐를 통해 웹 페이지를 순차적으로 방문. 각 페이지에서 h1 태그를 추출하고, 새로운 링크를 큐에 추가하여 크롤링을 확장
while queue: # 큐에 방문할 URL이 있는 동안 계속 반복
    if visited_count >= max_urls:
        print(f"Reached the limit of {max_urls} URLs. Stopping the crawler.")
        break

    current_url = queue.popleft() # 큐에서 다음 방문할 URL을 꺼내기
    if current_url in visited: # 이미 방문한 URL이면 다음 반복으로 건너뛰기
        continue

    visited.add(current_url) # 현재 URL을 방문한 URL 집합에 추가
    visited_count += 1

    if not can_fetch_url(current_url): # robots.txt 규칙을 확인하여 현재 URL이 접근 가능한지 확인
        print(f"Blocked by robots.txt: {current_url}") # 접근이 차단된 URL이면 메시지를 출력하고 다음 반복으로 건너뛰기
        continue

    response = get_response_with_retries(current_url)
    if response is None: # 응답이 성공적이지 않으면 메시지를 출력하고 다음 반복으로 건너뛰
        print(f"Failed to retrieve {current_url} after retries")
        continue

    soup = BeautifulSoup(response.text, 'html.parser') # 응답 내용을 BeautifulSoup으로 파싱

    titles = soup.find_all('h1') # 페이지 내의 모든 h1 태그를 찾
    for title in titles: # 각 h1 태그의 텍스트를 출력
        print(title.get_text())

    links = soup.find_all('a', href=True) # 페이지 내의 모든 a 태그에서 href 속성이 있는 링크를 찾
    for link in links:
        href = link['href']
        full_url = urljoin(current_url, href)  # 각 링크의 href 속성을 절대 경로로 변환하고, 큐에 추가
        if full_url not in visited and full_url.startswith('http'):
            queue.append(full_url)

#### BeautifulSoup
- BeautifulSoup은 HTML이나 XML 문서를 파싱하고, 파싱한 데이터에서 원하는 요소를 검색하고 추출하는 데 매우 유용한 도구입니다. 
- BeautifulSoup에서 객체를 찾는 주요 방법에는 find, find_all, select_one, select, find_parents, find_parent, find_next_sibling, find_previous_sibling 등이 있습니다.

검색 방식
- find, find_all: 태그 이름과 속성을 사용하여 요소를 검색합니다.
- select_one, select: CSS 선택자를 사용하여 요소를 검색합니다.

반환 결과:
- find: 첫 번째로 일치하는 요소를 반환합니다.
- find_all: 모든 일치하는 요소를 리스트로 반환합니다.
- select_one: 첫 번째로 일치하는 요소를 반환합니다.
- select: 모든 일치하는 요소를 리스트로 반환합니다.

표현력:
- select_one, select: 더 복잡하고 정교한 선택 조건을 지정할 수 있습니다. 예를 들어, CSS 선택자 문법을 사용하여 클래스, ID, 속성 등을 조합한 검색이 가능합니다.
- find, find_all: 단순한 태그 이름과 속성 조건에 기반한 검색이 주로 사용됩니다.



In [14]:
! pip install requests beautifulsoup4 urllib3 -q

html.parser vs. lxml
- 파이썬에서 HTML 및 XML 문서를 파싱(parsing)하는 라이브러리
- html.parser는 HTML 문서를 파싱하는 데에 적합한 파서. 파이썬의 기본 라이브러리로 제공되며 파이썬 내부적으로 구현되어 있으며, 외부 종속성이 없으므로 파이썬과 함께 설치되는 패키지만 사용할 수 있습니다.
- lxml은 C 언어로 작성된 파이썬 외부 라이브러리로서 HTML 및 XML 문서를 파싱하는 데에 적합하며, 파서 성능이 매우 우수합니다.
- HTML 문서를 파싱하는 경우에는 html.parser를 사용하는 것이 간단하고 편리하며, 대부분의 경우에는 충분한 성능을 제공합니다. 그러나 대용량의 XML 문서나 매우 복잡한 HTML 문서를 파싱해야 하는 경우에는 lxml을 사용하는 것이 더 효율적입니다.

In [7]:
from bs4 import BeautifulSoup

html_content = '<html><body><h1>Title</h1><p class="content">First paragraph.</p><p class="content">Second paragraph.</p></body></html>'
soup = BeautifulSoup(html_content, 'html.parser')
print(soup.prettify())

<html>
 <body>
  <h1>
   Title
  </h1>
  <p class="content">
   First paragraph.
  </p>
  <p class="content">
   Second paragraph.
  </p>
 </body>
</html>



In [8]:
# 첫 번째로 매칭되는 'p' 태그 찾기
first_p = soup.find('p')
print(first_p.get_text())  


First paragraph.


In [9]:
# 모든 'p' 태그 찾기
all_p = soup.find_all('p')
for p in all_p:
    print(p.get_text())

First paragraph.
Second paragraph.


In [10]:
# 클래스가 'content'인 첫 번째 'p' 태그 찾기
first_content_p = soup.select_one('p.content')
print(first_content_p.get_text())

First paragraph.


In [11]:
# 클래스가 'content'인 모든 'p' 태그 찾기
all_content_p = soup.select('p.content')
for p in all_content_p:
    print(p.get_text())

First paragraph.
Second paragraph.


In [12]:
specific_p = soup.find('p', class_='content')
specific_p

<p class="content">First paragraph.</p>

In [13]:
# 특정 'p' 태그의 모든 부모 태그 찾기
specific_p = soup.find('p', class_='content')
parents = specific_p.find_parents()
for parent in parents:
    print(parent.name)

body
html
[document]


In [14]:
# 특정 'p' 태그의 첫 번째 부모 태그 찾기
parent = specific_p.find_parent()
print(parent.name) 

body


In [34]:
# 특정 'p' 태그의 다음 형제 태그 찾기
next_sibling = specific_p.find_next_sibling()
print(next_sibling.get_text())

Second paragraph.


In [35]:
# 특정 'p' 태그의 이전 형제 태그 찾기
previous_sibling = specific_p.find_previous_sibling()
print(previous_sibling)

<h1>Title</h1>


In [36]:
# 특정 'p' 태그 다음에 위치한 모든 태그나 문자열 찾기
next_elements = specific_p.find_next()
print(next_elements)

<p class="content">Second paragraph.</p>


In [37]:
# 특정 'p' 태그 이전에 위치한 모든 태그나 문자열 찾기
previous_elements = specific_p.find_previous()
print(previous_elements) 

<h1>Title</h1>


In [66]:
import requests
from bs4 import BeautifulSoup

# 예시 네이버 뉴스 페이지 URL
url = 'https://news.naver.com'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
response

<Response [200]>

In [52]:
print(response)

<Response [200]>


response.content:
- response.content는 서버에서 반환된 응답을 바이트(byte) 문자열로 제공합니다.
- 주로 이미지, 파일 다운로드와 같은 바이너리 데이터를 다룰 때 사용됩니다.
- 인코딩과 상관없이 원본 그대로의 데이터를 가져오기 때문에, HTML 파싱을 할 때는 별도로 인코딩을 지정하지 않으면 기본 인코딩을 사용합니다.

response.text:
- response.text는 서버에서 반환된 응답을 유니코드 문자열로 제공합니다.
- requests 라이브러리는 response.text를 반환할 때, response.encoding에 지정된 인코딩을 사용하여 바이트 데이터를 유니코드 문자열로 디코딩합니다.
- 일반적인 텍스트 데이터, HTML, JSON 등의 처리를 할 때 유용합니다.

In [67]:
soup = BeautifulSoup(response.text, 'html.parser')
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<title id="browserTitleArea">네이버 뉴스</title>
<script>
	function isMobileDevice() {
		return /^.*(iPhone|iPod|iPad|Android).*/.test(navigator.userAgent);
	}
</script>
<script>
	(function () {
		try {
			if (isMobileDevice() && isAbleApplyPrefersColorScheme()) {
				
				document.querySelector("html").classList.add("DARK_THEME");
			}
		} catch(e) {}

		function isAbleApplyPrefersColorScheme() {
			
			if (window.matchMedia("(prefers-color-scheme)").matches === false) {
				return false;
			}

			var userAgent = navigator.userAgent;

			if (userAgent.indexOf("NAVER") > -1) {
				
				if (/.*NAVER\([a-zA-Z]*;\s[a-zA-Z]*;\s([0-9]*);/.test(userAgent)) {
					return Number(RegExp.$1) >= 1000;
				}
			} else {
				
				return document.cookie.indexOf("NSCS=1") > -1;
			}

			return false;
		}
	})();
</script>
<script>
		var g_ssc = 'news.v3_media' || null;
		</script>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,maximum-s

In [57]:
# 'title' 태그만으로 요소 찾기
news_titles = soup.find('title')
news_titles

<title id="browserTitleArea">네이버 뉴스</title>

In [58]:
# title 태그와 id 속성을 이용하여 원하는 요소를 찾는 방법
title_element = soup.find('title', id='browserTitleArea')
title_element

<title id="browserTitleArea">네이버 뉴스</title>

In [59]:
cats = soup.find_all(class_='Nitem_link_menu')
for idx, cat in enumerate(cats):
    print(f"{idx + 1}: {cat.get_text().strip()}")

1: 언론사별
2: 정치
3: 경제
4: 사회
5: 생활/문화
6: IT/과학
7: 세계
8: 랭킹
9: 신문보기
10: 오피니언
11: TV
12: 팩트체크
13: 알고리즘 안내
14: 정정보도 모음


In [60]:
# CSS 선택자를 사용하여 title 태그 중 ID가 browserTitleArea인 요소를 검색
title_element = soup.select_one('title#browserTitleArea')
title_element

<title id="browserTitleArea">네이버 뉴스</title>

In [62]:
item_element = soup.select_one('span.Nicon_service')
item_element


<span class="Nicon_service">뉴스</span>

In [61]:
cats = soup.select('.Nitem_link_menu')
for idx, cat in enumerate(cats):
    print(f"{idx + 1}: {cat.get_text().strip()}")

1: 언론사별
2: 정치
3: 경제
4: 사회
5: 생활/문화
6: IT/과학
7: 세계
8: 랭킹
9: 신문보기
10: 오피니언
11: TV
12: 팩트체크
13: 알고리즘 안내
14: 정정보도 모음


In [68]:
titles = soup.select('.cjs_t')
for idx, title in enumerate(titles):
    print(f"{idx + 1}: {title.get_text().strip()}")

1: [속보] '훈련병 사망' 얼차려 지시 중대장·부중대장 피의자 신분 첫 소환조사
2: BTS 진에게 기습 뽀뽀한 팬들, 성추행 혐의로 고발 당했다
3: 김제 페인트 공장 화재.. 폐기물 보관 드럼 100개 소실
4: 전두환 보도지침 언급한 정동영 "세상 거꾸로 와버려"
5: “빈집 고쳐 월 1만 원 임대라니, 더 없나요?”
6: 윤 대통령 부부가 선물 받은 투르크 국견…한남동 관저로
7: 법정 나온 이재명 "대북송금 기소, 희대의 조작 사건 될 것"
8: 러 검찰, 간첩 혐의 WSJ 기자 기소…"러 전차 정보 불법 수집"
9: "엔비디아와 어깨 나란히"...시총 1조달러 주인공될 '이곳'
10: 국민 60%가 ‘석유 가능성’ 안 믿는 이유 [김은지의 뉴스IN]
11: '김건희 논문 진상규명' 공약한 교수 숙대 총장 최다득표
12: '자본금 미달' 스테이지엑스…제4이통 자격 취소
13: "김여사, 내돈내산 디올백·화장품 돌려줘" 서울의소리 기자 경찰 출석
14: "혼자 산지 오래됐다던 아내에게 숨겨둔 아이가 있었습니다"
15: Korea's KTX makes first overseas export deal with Uzbekistan
16: “나 구청 직원이야…망하게 해줄게”
17: '김만배와 돈거래' 한국일보 전직 간부, 해고 유지
18: Science Ministry to cancel license of Stage X as new mobile carrier
19: 혼자 대선까지 가버린 한동훈
20: 때이른 '불볕 더위'에 온열질환자 급증...주의할 점은?
21: 인천공항 등 100여곳 '발칵' 뒤집혀..."폭탄 터뜨린다" 테러 협박 메일
22: 장항선 광천-보령 청소구간서 화물열차 고장…수송원 2명 부상
23: [현장] 인공지능이 립스틱 바르는 걸 돕는다, ‘손떨림’ 장애인 위해
24: "너가 먼저 꼬리쳤지"…담당 경찰, 신상 털렸다
25: 33살 연하와 사랑…70살 할머니 홍학이 알을 낳았다
26: 액트지오 결론 교차검증 전문가, 석유공사 동해탐사팀장 

string vs. get_text() vs. text
- BeautifulSoup 객체에서 text 속성, get_text() 메서드, string 속성은 모두 HTML 또는 XML 문서에서 텍스트 데이터를 추출하는 데 사용
- text 속성은 해당 태그에서 모든 텍스트 데이터를 가져오며,
- get_text() 메서드도 동일한 결과를 반환합니다.
- string 속성은 해당 태그에서 첫 번째로 발견된 문자열 데이터만 가져옵니다.

| Method      | 특징                                                                               | 사용 사례                                      |
|-------------|----------------------------------------------------------------------------------|-----------------------------------------------|
| `string`    | 자식 텍스트 노드가 하나인 경우에만 해당 텍스트 반환, 여러 노드가 있으면 `None` 반환                | 단일 텍스트 노드만 있는 요소의 텍스트를 추출할 때      |
| `get_text()`| 요소 및 모든 하위 요소의 텍스트를 모두 포함하여 문자열로 반환, `separator` 및 `strip` 옵션 사용 가능 | 요소 내 모든 텍스트를 추출하고, 구분자 및 공백 처리가 필요한 경우 |
| `text`      | 요소 및 모든 하위 요소의 텍스트를 모두 포함하여 문자열로 반환, 추가 매개변수 없음                  | 요소 내 모든 텍스트를 단순하게 추출할 때                   |

string
- 정의: string 속성은 BeautifulSoup 객체의 자식 노드 중 하나의 텍스트 노드만 반환. 해당 요소의 직접적인 텍스트 콘텐츠가 하나일 때만 유용.
- 특징:
요소가 하나의 자식 텍스트 노드를 가지고 있는 경우에만 그 텍스트를 반환.
여러 자식 요소가 있거나 텍스트 노드가 여러 개 있는 경우 None을 반환.

In [44]:
from bs4 import BeautifulSoup

html = """
<div class="example">
    Single text node
</div>
<div class="example">
    <p>Multiple</p> text <span>nodes</span>
</div>
"""

soup = BeautifulSoup(html, 'html.parser')
single_text_node = soup.find('div', class_='example').string
multiple = soup.find_all('div', class_='example')

multiple_text_nodes = soup.find_all('div', class_='example')[1].string

print(single_text_node)  # Output: Single text node
print(multiple,'\n')
print(multiple_text_nodes)  # Output: None


    Single text node

[<div class="example">
    Single text node
</div>, <div class="example">
<p>Multiple</p> text <span>nodes</span>
</div>] 

None


get_text()
- 정의: get_text() 메서드는 요소 및 모든 하위 요소의 텍스트를 모두 추출하여 하나의 문자열로 반환.
- 특징:
요소 내의 모든 텍스트를 포함하여 반환.
기본적으로 하위 요소 사이에 공백을 추가하지만, separator 매개변수를 사용하여 구분자를 지정할 수 있다.
strip=True 옵션을 사용하여 앞뒤 공백을 제거할 수 있다.

In [46]:
from bs4 import BeautifulSoup

html = """
<div class="example">
    Single text node
</div>
<div class="example">
    <p>Multiple</p> text <span>nodes</span>
</div>
"""

soup = BeautifulSoup(html, 'html.parser')

# 기본적으로 `get_text()` 메서드는 공백을 구분자로 사용합니다.
# strip=True 옵션을 사용하여 앞뒤 공백을 제거하고 텍스트를 추출
single_text_node = soup.find('div', class_='example').get_text(strip=True)
multiple_text_nodes = soup.find_all('div', class_='example')[1].get_text(strip=True)

print("Default separator:")
print(single_text_node)  # Output: Single text node
print(multiple_text_nodes)  # Output: Multiple text nodes



Default separator:
Single text node
Multipletextnodes


In [47]:
# `separator` 매개변수를 사용하여 구분자를 지정합니다.
single_text_node_with_separator = soup.find('div', class_='example').get_text(strip=True)
multiple_text_nodes_with_separator = soup.find_all('div', class_='example')[1].get_text(separator=' | ', strip=True)

print("\nCustom separator (' | '):")
print(single_text_node_with_separator)  # Output: Single text node
print(multiple_text_nodes_with_separator)  # Output: Multiple | text | nodes


Custom separator (' | '):
Single text node
Multiple | text | nodes


text
- 정의: text 속성은 요소 및 모든 하위 요소의 텍스트를 모두 포함하는 문자열을 반환. get_text() 메서드와 동일한 기능을 한다.
- 특징:
get_text()와 거의 동일한 결과를 제공.
get_text()와 다르게 추가 매개변수(separator, strip)를 사용할 수 없다.

In [48]:
from bs4 import BeautifulSoup

html = """
<div class="example">
    Single text node
</div>
<div class="example">
    <p>Multiple</p> text <span>nodes</span>
</div>
"""

soup = BeautifulSoup(html, 'html.parser')
single_text_node = soup.find('div', class_='example').text
multiple_text_nodes = soup.find_all('div', class_='example')[1].text

print(single_text_node)  # Output: Single text node
print(multiple_text_nodes)  # Output: Multiple text nodes


    Single text node


Multiple text nodes



정규표현식을 이용한 크롤링 방법 

HTML 가져오기:
- requests 라이브러리를 사용하여 웹 페이지의 HTML을 가져옵니다.
HTML 파싱:
- BeautifulSoup을 사용하여 HTML 문서를 파싱합니다.
정규표현식 사용:
- re 모듈을 사용하여 특정 패턴을 가진 텍스트를 추출합니다.

In [60]:
import requests
from bs4 import BeautifulSoup
import re

html = """
<ul>
  <li>Email: example@example.com</li>
  <li>Contact: contact@sample.org</li>
</ul>
"""
# 이메일 주소 추출
email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
emails = re.findall(email_pattern, html)
print(emails)


['example@example.com', 'contact@sample.org']


In [62]:
html = '''
<html>
<body>
<div>
Hello,world!
</div>
<div>
<p>
Hello,<b>world!</b>
</p>
</div>
</body>
</html>
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
bs = soup.body.text
print(bs)



Hello,world!



Hello,world!





In [63]:
import re
text = re.findall('[^\s]+',bs)
for t in text:
    print(t)
# print(re.findall('[A-Za-z!]+',bs))

Hello,world!
Hello,world!


In [64]:
li = ''
for t in text:
    li += t
    li += ' '
print(li)

Hello,world! Hello,world! 


In [99]:
from bs4 import BeautifulSoup
import re # 정규 표현식을 사용할 때
html = """
<ul>
  <li><a href="hoge.html">hoge</li>
  <li><a href="https://example.com/fuga">fuga*</li>
  <li><a href="https://example.com/foo">foo*</li>
  <li><a href="http://example.com/aaa">aaa</li>
</ul>
"""
soup = BeautifulSoup(html, "html.parser")
# 정규 표현식으로 href에서 https인 것 추출하기
li = soup.find_all(href=re.compile(r"^https://"))
for e in li: print(e.attrs['href'])

https://example.com/fuga
https://example.com/foo


#### urllib + bs

In [69]:
import urllib.request as rq

url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100'

html = rq.urlopen(url)
bs = BeautifulSoup(html, 'html.parser')
print(bs)

<!DOCTYPE html>

<html data-useragent="Python-urllib/3.10" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport">
<meta content="정치 : 네이버 뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/section/100" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png" property="og:image"/>
<meta content="국회, 행정, 국방, 외교 등 정치 분야 뉴스 제공" property="og:description"/>
<meta content="네이버" property="og:article:author"/>
<meta content="summary" name="twitter:card"/>
<meta content="정치 : 네이버 뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pst

In [70]:
text1 = bs.find('p')
print(text1)

<p class="sa_head_layer_p">각 헤드라인의 기사와 배열 순서는 개인화를 반영해 자동 추천되어 제공됩니다. 기사 수량이 표기된 기사 우측 하단의 파란색 아이콘을 클릭하면 기사묶음을 확인할 수 있고 기사묶음과 기사묶음 타이틀도 기사 내용을 기반으로 <strong>자동 추출</strong>됩니다.</p>


In [71]:
text10 = bs.find_all('p')
for t in text10:
    print(t.text)

각 헤드라인의 기사와 배열 순서는 개인화를 반영해 자동 추천되어 제공됩니다. 기사 수량이 표기된 기사 우측 하단의 파란색 아이콘을 클릭하면 기사묶음을 확인할 수 있고 기사묶음과 기사묶음 타이틀도 기사 내용을 기반으로 자동 추출됩니다.
AiRS 추천으로 구성된 뉴스를 제공합니다.
각 언론사의 가장 많이 본 기사 1건을 제공합니다.
33살 연하와 사랑…70살 할머니 홍학이 알을 낳았다
[단독]"당첨되면 5억 로또"…전세가 6억 이 아파트, 6억 '줍줍' 떴다
"엔비디아와 어깨 나란히"...시총 1조달러 주인공될 '이곳'
"네가 먼저 꼬리 쳤지" 폭언한 밀양 경찰관 '실명·얼굴' 다 털렸다
중국팬에 돈쭐난 싱가포르 골키퍼 "돈 좀 그만 보내라" 호소, 왜
폭염에 얼음 쌓아 두고 수업‥40도 넘는 폭염에 펄펄 끓는 중국
"터지기 일보 직전" 175년 만에 최악…'초비상' 걸렸다
"입사하면 200만원 드립니다" 입사축하금 내건 이 회사
[씬속뉴스]푸바오 '작은 할부지' 송바오 "너는 정말 좋은 판다라는걸 알아" 응원
"진 뽀뽀 시도한 팬들 엄벌해야"...경찰에 민원 접수
반발에 놀랐나… 서울의대 교수들 “중증·희귀질환 환자께 죄송”
"중국이 중국했다"? 중국, 축구 패배 분풀이를 손흥민에게? [뉴스와이드]
국민의힘 "김정숙 여사 타지마할, 청와대 급박한 요청 있었다"
[단독] "완전 단전" 묵살 12분 뒤 감전사…유족에 남겨진 건 찢긴 작업복뿐
“한국에 들어오면 난리나겠네”…‘3억통’ 팔린 연기 안나는 담배의 정체
“60만원 주고 샀는데, 팔찌로 써요”…인기 ‘뚝’ 급해진 삼성, 눈물의 ‘결단’
“삼겹살 ‘한 캔’ 주세요” 뚜껑 따자…“맥주 같아” MZ들 환호
14억 아파트 8억 됐다...반토막 속출 이곳, 지금 사야 될까
“내일 아침 폭탄 터질 것” 인천공항 등 공기관 100곳에 날아든 이메일
"너가 먼저 꼬리쳤지"…담당 경찰, 신상 털렸다
"삼겹살 한 캔, 목살 두 캔 주세요"…출시되자마자 캠핑족 '최애템' 된 '이것'
따릉이 27대 하천에 내동댕이…20대 용의자 

In [73]:
bs.find('p',class_="sa_head_layer_p")

<p class="sa_head_layer_p">각 헤드라인의 기사와 배열 순서는 개인화를 반영해 자동 추천되어 제공됩니다. 기사 수량이 표기된 기사 우측 하단의 파란색 아이콘을 클릭하면 기사묶음을 확인할 수 있고 기사묶음과 기사묶음 타이틀도 기사 내용을 기반으로 <strong>자동 추출</strong>됩니다.</p>

In [74]:
text2 = bs.find('p').string
print(text2)

None


In [75]:
print(bs.find('title'))

<title>정치 : 네이버 뉴스</title>


In [76]:
print(bs.find('title').string)

정치 : 네이버 뉴스


In [77]:
text3 = bs.find('p').get_text()
print(text3)

각 헤드라인의 기사와 배열 순서는 개인화를 반영해 자동 추천되어 제공됩니다. 기사 수량이 표기된 기사 우측 하단의 파란색 아이콘을 클릭하면 기사묶음을 확인할 수 있고 기사묶음과 기사묶음 타이틀도 기사 내용을 기반으로 자동 추출됩니다.


#### requests + bs

인코딩 에러를 해결

 chardet.detect(response.content)['encoding']은 response.content의 인코딩 방식을 자동으로 감지하여 반환하며
 이 값을 encoding 변수에 저장한 후, response.content를 이 encoding 방식으로 디코딩하여 html 변수에 저장하고 출력

In [78]:
! pip install chardet

   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   ---------------------------------------- 199.4/199.4 kB 5.9 MB/s eta 0:00:00


In [1]:
import requests
import chardet

# HTTP 요청에서 사용될 헤더 정보를 설정합니다.
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

# 웹사이트에서 스크래핑할 URL을 지정합니다.
url = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=100'

# 지정된 url에 HTTP GET 요청을 전달
response = requests.get(url, headers=headers)

# HTTP 요청이 성공적으로 전달되었다면, 웹사이트의 HTML 코드를 출력합니다.
if response.status_code == 200:
    encoding = chardet.detect(response.content)['encoding']
    # print(response.content.decode(encoding)) # 웹사이트의 HTML 코드를 포함하는 이진(binary) 데이터를 반환
    print(response.text) # 웹사이트의 HTML 코드를 문자열 형태로 반환
else:
    print("HTTP request failed.")

<!doctype html>
<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36">
	<head>
		<meta charset="utf-8">
		<meta http-equiv="X-UA-Compatible" content="IE=edge">
		<meta name="referrer" contents="always">
		<meta http-equiv="refresh" content="600">
		<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" />
		<meta property="og:title" content="정치 : 네이버 뉴스">
		<meta property="og:type" content="website">
		<meta property="og:url" content="https://news.naver.com/section/100">
		<meta property="og:image" content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png">
		<meta property="og:description" content="국회, 행정, 국방, 외교 등 정치 분야 뉴스 제공">
		<meta property="og:article:author" content="네이버">
		<meta name="twitter:card" content="summary">
		<meta name="twitter:title" content="정치 : 네이버 뉴스">
		<meta nam

In [ ]:
# # 한글만 출력
import re
texts = response.text
result = re.findall('[가-힣]+',texts)
print(' '.join(result))

In [5]:
import requests
from bs4 import BeautifulSoup

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

url = "https://serieson.naver.com/v3/movie/ranking/realtime"

# HTTP GET 요청을 보냅니다.
response = requests.get(url, headers=headers)

# HTTP 요청이 성공적으로 전달되었다면, 웹사이트의 HTML 코드를 가져옵니다.
if response.status_code == 200:
    # HTML 코드를 파싱합니다.
    soup = BeautifulSoup(response.text, "html.parser")
# print(soup)
soup.find("span", class_="Title_title__s9o0D").text

'인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정)'

In [6]:
list = soup.find_all("span", class_="Title_title__s9o0D")
for i, l in enumerate(list[:10]):
    print(f'{i+1}: {l.text}')

1: 인사이드 아웃(패키지상품 : 더빙판 + 부가영상 추가증정)
2: 악마와의 토크쇼
3: 파묘
4: 블랙북
5: 소울메이트
6: 매드맥스: 분노의 도로
7: 파묘
8: 귀공자
9: 콰이어트 플레이스 2
10: 너의 이름은.


### CSS 선택자
- 원하는 정보만 선별하여 수집하고 싶을 때 css선택자를 활용할 수 있음
- (CSS 선택자 설명 추가)
- F12 >> 수집하고 싶은 부분 클릭 >> 태그 선택 >> copy Selector
- BeautifulSoup의 select_one, select 활용

In [3]:
# 다음 정치 카테고리 뉴스 크롤링
import requests as rq
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
url = 'https://news.daum.net/politics#1'
r = rq.get(url, headers=headers)
html = r.text
soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('.link_txt')
body ='\n'.join([f'{i+1}. {line.text}' for i, line in enumerate(lines)])
print(body)

1. 軍 "北 휴전선 인근 담벼락 기초 공사"...국경선 긋나?
2. "결국 장관 지시잖아요" "예"…유재은 법정 증언 집중 분석
3. 연일 檢 때리는 민주…“대북송금 수사, 역사에 오점으로 남을 것”[이런정치]
4. 지하철 잠든 게 정치쇼? 이준석 “상계동에선 이슈 아냐, 옆 자리 분께 죄송”
5. 원구성 강경 대치…"전면 백지화" vs "17일 완료"
6. “애완견 망언” “희대의 조작수사”…이재명 기소 공방
7. 6·15 실천 남측위, '평화연대'로 새 출발...명칭 변경
8. 고위 당정대, 저출생·전력수급 대책 및 재해대응 논의
9. 태국 공군참모장, 전쟁기념관 방문…"참전국 군인 희생 기억해줘 영광"
10. [뉴스1번지] 여 전당대회 한동훈 등판 임박…당권주자들 경쟁 시작
11. 정점 향하는 북러 밀착...북한 위협 막을 한국 무기는?
12. 尹, 사마르칸트行..우즈벡 국빈방문 마무리
13. 국회, '집단 휴진'에 손 내밀었지만...與 따로, 野 따로
14. ‘전시(戰時) 같은 토요일’... 여야 논평 따져보니
15. 이재명 추가 기소…野 '특검·법사위' 대응, 與 '특위' 맞불
16. 민주 "대북송금 기소, 이재명 죽이려는 희대의 조작"
17. 연일 檢 때리는 민주…“대북송금 수사, 역사에 오점으로 남을 것”
18. 러 갔던 사회안전성대표단 귀국…화성지구 골조공사 마감단계
19. 이쯤되면 액트지오 사태-다급해진 이재명-콩 세는 한동훈
20. 어느새 청원 6만명 돌파...與 ‘금투세 폐지’ 당론 발의
21. 헌법재판소가 대북전단 살포를 허용했다고?
22. 한미 학생 동고동락…‘평화 통일’ 한마음
23. 결혼·출산 기피…‘계약 결혼’도 등장
24. ‘첫치기’서 ‘정착지’로…북한식 골프 소개 외
25. 남북, 심리전 숨고르기…전단 단속 엇박자 외
26. 뉴스홈
27. 사회
28. 정치
29. 경제
30. 국제
31. 문화
32. IT
33. 포토
34. 제휴 언론사
35. 배열이력
36. 전체뉴스
37. 연재
38. 팩트체크


In [2]:
# 개발자 도구 copy.selector
import requests as rq
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
url = 'https://news.daum.net/politics#1'
r = rq.get(url, headers=headers)
html = r.text
soup = BeautifulSoup(html, 'html.parser')

titles = soup.select('body > div > main > section > div.main-sub > div > ul > li > strong > a')
for i, title in enumerate(titles, start=1):
    print(f"{i}. {title.get_text().strip()}")

1. [스트레이트 예고] 22대 국회 긴급점검-국민없는 정치, 정치없는 국민
2. '김건희 논문 표절' 규명 약속한 1위 숙대총장 후보…조국 "중전마마 위해 이 분 주저앉힐까?"
3. 대통령실 “종부세 사실상 전면 폐지 바람직…상속세 전면 개편”
4. '부대 특성 살렸더니…' 공군 스페이스챌린지 인기, 벌써 작년 방문객 육박
5. 野, 의대 교수 비대위 면담...與, 고위 당정대 회의
6. 몽골 황제 주치의 지낸 ‘몽골의 한국인 슈바이처’ 이태준기념관 울란바토르에 건립
7. 성태윤 정책실장 "다주택자도 총합 적으면 종부세 폐지가 바람직"
8. 최근 3년간 경북 중·고생 200여 명 사이버 도박···"1억 5,500만 원 규모"
9. “이게 신개념 K-도하”… 한국형 자주도하장비 ‘수룡’ 실전 배치됐다 [박수찬의 軍]
10. [단독]석유공사, 이미 1월 ‘동해 심해’ 탐사 시추 이사회 의결…대통령 직접 브리핑 왜?


In [7]:
# 한글과 숫자만 출력, 콤마 제거
import re

soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('body > div > main > section > div.main-sub > div > ul > li > strong > a')

for i, tag in enumerate(lines, start=1):
    text = tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+', text)
    print(f"{i}. {' '.join(matches)}")

# print(' '.join(texts))

1. 스트레이트 예고 22대 국회 긴급점검 국민없는 정치 정치없는 국민
2. 김건희 논문 표절 규명 약속한 1위 숙대총장 후보 조국 중전마마 위해 이 분 주저앉힐까
3. 대통령실 종부세 사실상 전면 폐지 바람직 상속세 전면 개편
4. 부대 특성 살렸더니 공군 스페이스챌린지 인기 벌써 작년 방문객 육박
5. 의대 교수 비대위 면담 고위 당정대 회의
6. 몽골 황제 주치의 지낸 몽골의 한국인 슈바이처 이태준기념관 울란바토르에 건립
7. 성태윤 정책실장 다주택자도 총합 적으면 종부세 폐지가 바람직
8. 최근 3년간 경북 중 고생 200여 명 사이버 도박 1억 5500만 원 규모
9. 이게 신개념 도하 한국형 자주도하장비 수룡 실전 배치됐다 박수찬의
10. 단독 석유공사 이미 1월 동해 심해 탐사 시추 이사회 의결 대통령 직접 브리핑 왜


In [10]:
# pandas 데이터프레임으로 변환 후 csv 파일로 저장
import re
import pandas as pd

soup = BeautifulSoup(html, 'html.parser')

lines = soup.select('body > div > main > section > div.main-sub > div > ul > li > strong > a')
results = []
for i, tag in enumerate(lines, start=1):
    text = tag.get_text().strip().replace(',', '')
    matches = re.findall('[가-힣0-9]+', text)
    result = ' '.join(matches)
    results.append({'Index':i, 'Text':result})
    
df = pd.DataFrame(results)

df.to_csv('news.csv', index=False)
df = pd.read_csv('news.csv')
df

,Index,Text
0,1,스트레이트 예고 22대 국회 긴급점검 국민없는 정치 정치없는 국민
1,2,김건희 논문 표절 규명 약속한 1위 숙대총장 후보 조국 중전마마 위해 이 분 주저앉힐까
2,3,대통령실 종부세 사실상 전면 폐지 바람직 상속세 전면 개편
3,4,부대 특성 살렸더니 공군 스페이스챌린지 인기 벌써 작년 방문객 육박
4,5,의대 교수 비대위 면담 고위 당정대 회의
5,6,몽골 황제 주치의 지낸 몽골의 한국인 슈바이처 이태준기념관 울란바토르에 건립
6,7,성태윤 정책실장 다주택자도 총합 적으면 종부세 폐지가 바람직
7,8,최근 3년간 경북 중 고생 200여 명 사이버 도박 1억 5500만 원 규모
8,9,이게 신개념 도하 한국형 자주도하장비 수룡 실전 배치됐다 박수찬의
9,10,단독 석유공사 이미 1월 동해 심해 탐사 시추 이사회 의결 대통령 직접 브리핑 왜


- req.status_code: HTTP 상태 코드를 반환합니다. 예를 들어, 200은 성공을 의미하고, 404는 페이지를 찾을 수 없음을 의미합니다.
- req.text: 응답 본문을 문자열로 반환합니다. HTML 페이지의 전체 소스를 볼 수 있습니다.
- req.content: 응답 본문을 바이트 형태로 반환합니다.
- req.url: 요청된 최종 URL을 반환합니다.
- req.headers: 응답 헤더를 반환합니다.

In [12]:
# 검색 키워드와 뉴스 기사 수 입력받기
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

query = input('검색 키워드를 입력하세요 : ')
news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))
print(query)
query = query.replace(' ', '+')
print(query)

news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

# format(query) 메서드는 URL 템플릿의 {} 부분을 query로 대체
req = requests.get(news_url.format(query)) 
print(req)


ai
ai
<Response [200]>


In [13]:
req.url

'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=ai'

In [37]:
html = req.text
soup = BeautifulSoup(html, 'html.parser')
links = soup.select('.news_tit')
# print(links)
for link in links:
    title = link.text
    url = link.attrs['href']
    print(title, url)

SKT, 통신·기술 연합체 TM포럼서 AI 동맹 확장 나서 https://www.yna.co.kr/view/AKR20240616015900017?input=1195m
한·일·중 교육장관, 4년5개월만 서울서 회담…AI교육·유학 논의 https://www.newsis.com/view/NISX20240614_0002773558
전국 최초 AI‧메타버스 영화제 구미에서 개최 https://www.wikitree.co.kr/articles/959776
'AI 돌풍' 반도체 호황 지속…5월 ICT 75억달러 흑자 https://www.news1.kr/articles/5448155
"100년 내 인간 멸종 가능성 99.9%"… AI 석학들의 경고 http://weekly.chosun.com/news/articleView.html?idxno=35287
생성형AI 혁명...누구나 혁신가가 될 수 있다[허태윤의 브랜드 스토리] https://economist.co.kr/article/view/ecn202406100041
AI로 만든 영화끼리 경쟁?...부천국제영화제의 색다른 도전 https://www.ytn.co.kr/_ln/0106_202406160224089159
아마존, AI 스타트업 지원에 2.3억 달러 투자 https://zdnet.co.kr/view/?no=20240616110326
한국IBM “생성형 AI 시대, ‘자동화’는 선택 아닌 필수” http://www.segye.com/newsView/20240613517435?OutUrl=naver
[정책 인사이트] ‘AI 공무원’ 회의록 작성에 민원 응대까지 바쁘다 바빠 https://biz.chosun.com/topics/topics_social/2024/06/16/CEXASWGFYJBBPKJDKDGJOST3RI/?utm_source=naver&utm_medium=original&utm_campaign=biz


In [38]:
# urllib 사용
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup

baseUrl = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query='
plusUrl = input('검색어를 입력하세요: ')
url = baseUrl + urllib.parse.quote_plus(plusUrl)

html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html,'html.parser')

title = soup.find_all(class_='news_tit')

for i in title:
    print(i.attrs['title'])
    print(i.attrs['href'])
    print()

SKT, 통신·기술 연합체 TM포럼서 AI 동맹 확장 나서
https://www.yna.co.kr/view/AKR20240616015900017?input=1195m

한·일·중 교육장관, 4년5개월만 서울서 회담…AI교육·유학 논의
https://www.newsis.com/view/NISX20240614_0002773558

전국 최초 AI‧메타버스 영화제 구미에서 개최
https://www.wikitree.co.kr/articles/959776

'AI 돌풍' 반도체 호황 지속…5월 ICT 75억달러 흑자
https://www.news1.kr/articles/5448155

"100년 내 인간 멸종 가능성 99.9%"… AI 석학들의 경고
http://weekly.chosun.com/news/articleView.html?idxno=35287

생성형AI 혁명...누구나 혁신가가 될 수 있다[허태윤의 브랜드 스토리]
https://economist.co.kr/article/view/ecn202406100041

AI로 만든 영화끼리 경쟁?...부천국제영화제의 색다른 도전
https://www.ytn.co.kr/_ln/0106_202406160224089159

아마존, AI 스타트업 지원에 2.3억 달러 투자
https://zdnet.co.kr/view/?no=20240616110326

한국IBM “생성형 AI 시대, ‘자동화’는 선택 아닌 필수”
http://www.segye.com/newsView/20240613517435?OutUrl=naver

[정책 인사이트] ‘AI 공무원’ 회의록 작성에 민원 응대까지 바쁘다 바빠
https://biz.chosun.com/topics/topics_social/2024/06/16/CEXASWGFYJBBPKJDKDGJOST3RI/?utm_source=naver&utm_medium=original&utm_campaign=biz



In [44]:
# 여러 페이지 크롤링을 위한 url 생성
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import re
from datetime import datetime

# 현재 날짜와 시간 포맷팅
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':', '시') + '분'

# 사용자로부터 검색 키워드 입력 받기
query = input('검색 키워드를 입력하세요 : ')
encoded_query = requests.utils.quote(query)  # URL 인코딩

# 기본 URL 설정
base_url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={encoded_query}'

# 페이지 URL 목록을 반복문을 사용해 생성
page_urls = [base_url]  # 첫 번째 페이지 URL
for page in range(2, 11):  # 2 페이지부터 10 페이지까지 URL 생성
    start = (page-2) * 15 + 1 
    page_url = f'https://search.naver.com/search.naver?nso=&page={page}&query={encoded_query}&sm=tab_pge&start={start}&where=web'
    page_urls.append(page_url)
print(page_urls)

['https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90', 'https://search.naver.com/search.naver?nso=&page=2&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_pge&start=1&where=web', 'https://search.naver.com/search.naver?nso=&page=3&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_pge&start=16&where=web', 'https://search.naver.com/search.naver?nso=&page=4&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_pge&start=31&where=web', 'https://search.naver.com/search.naver?nso=&page=5&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_pge&start=46&where=web', 'https://search.naver.com/search.naver?nso=&page=6&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_pge&start=61&where=web', 'https://search.naver.com/search.naver?nso=&page=7&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_pge&start=76&where=web', 'https://search.naver.com/search.naver?nso=&page=8&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_pge&start=91&where=

In [45]:
# 여러 페이지 내용 출력
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import re
from datetime import datetime


# 사용자로부터 검색 키워드 입력 받기
query = input('검색 키워드를 입력하세요 : ')
encoded_query = requests.utils.quote(query)  # URL 인코딩

# 기본 URL 설정
base_url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={encoded_query}'

# 페이지 URL 목록을 반복문을 사용해 생성
page_urls = [base_url]  # 첫 번째 페이지 URL
for page in range(2, 11):  # 2 페이지부터 10 페이지까지 URL 생성
    start = (page-2) * 15 + 1 
    page_url = f'https://search.naver.com/search.naver?nso=&page={page}&query={encoded_query}&sm=tab_pge&start={start}&where=web'
    page_urls.append(page_url)

news_dict = {}
idx = 0

print('크롤링 중...')

# 각 페이지에서 뉴스 제목과 링크 크롤링
for url in page_urls:
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

    lists = soup.select('.api_txt_lines.total_dsc.type_3lines')

    for n in lists:
        news_dict[idx] = {'url': n.get('href')}
        idx += 1

print('크롤링 완료')

# 데이터프레임 변환
print('데이터프레임 변환')
news_df = DataFrame(news_dict).T

# 결과 출력
print(news_df)



크롤링 중...
크롤링 완료
데이터프레임 변환
                                                   url
0    https://namu.wiki/w/%EC%82%BC%EC%84%B1%EC%A0%8...
1                 https://blog.naver.com/samsung_korea
2                         https://news.samsung.com/kr/
3                        https://www.samsungsvc.co.kr/
4             https://www.facebook.com/SamsungNewsroom
..                                                 ...
105       https://www.samsung.com/sec/smartthings/app/
106  https://www.instagram.com/explore/tags/%EC%82%...
107  https://www.samsung.com/sec/air-conditioners/a...
108                        https://samsunglabor.co.kr/
109                https://www.samsung.com/sec/md-inv/

[110 rows x 1 columns]


In [49]:
# 5번 스크롤 다운
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os


query = input('검색 키워드를 입력하세요 : ')

news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

driver = webdriver.Chrome()

driver.get(news_url.format(query))

time.sleep(1)

for i in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

items = soup.select('.news_tit')

for e, item in enumerate(items, 1):
    print(f'{e} : {item.text}')
    
driver.quit()

1 : [단독] LG전자 'GPU 팜' 구축…“AI기술로 고객경험 혁신”
2 : SKT, 통신·기술 연합체 TM포럼서 AI 동맹 확장 나서
3 : '연일 무더위' 삼성·LG, 똑똑한 AI에어컨 맞대결
4 : 한·일·중 교육장관, 4년5개월만 서울서 회담…AI교육·유학 논의
5 : 영상 예술로 파고든 인공지능...국내 최초 'AI·메타버스 영화제' 열려
6 : 'AI 돌풍' 반도체 호황 지속…5월 ICT 75억달러 흑자
7 : SK텔레콤, 글로벌 연합체 ‘TM포럼’서 AI 동맹 확장
8 : "AI시대, 소통·창의력 지닌 리더 필요…인문학 더 중요해질 것"
9 : '원더랜드' 공유 목소리가 AI? 김태용 감독도 똑같아서 놀랐다
10 : "AI 열풍에 전기수요 폭증"…변압기업체 '거침없는 질주'
11 : 韓 1040억 vs 美 22조…韓AI 초라한 투자 성적표
12 : '세계 최초' AI 미인대회, 외모 중요하지 않아…왜?
13 : "100년 내 인간 멸종 가능성 99.9%"… AI 석학들의 경고
14 : '원더랜드' 공유 목소리가 AI? 김태용 감독도 똑같아서 놀랐다
15 : 'AI 호황'에 ICT 수출액 두달 연속 30%대↑…내수도 견인할까
16 : "서울대 하영선 교수입니다"…AI, 한달만에 77세 노교수 됐다
17 : 메타 "유럽서 '메타 AI' 당분간 출시하지 않을 것"
18 : "AI 발전으로 사이버 공격 위협도 증대…국제협력 필요"
19 : 뉴욕증시, 'AI 파도타기' 이번은 어도비…나스닥 역대 최고 마감
20 : 뉴욕증시, ‘AI 열풍’에 나스닥 역대 최고 마감
21 : 생성형AI 혁명...누구나 혁신가가 될 수 있다[허태윤의 브랜드 스토리]
22 : AI로 만든 영화끼리 경쟁?...부천국제영화제의 색다른 도전
23 : 아마존, AI 스타트업 지원에 2.3억 달러 투자
24 : AI로 불법촬영 감지…성동구, 안전한 공중화장실 조성
25 : LG전자, 스페인에 AI가전 체험공간 '어나더 한옥' 공개
26 : 한국IBM “생성형 AI 시대, ‘자동화’는 

In [59]:
# 검색한 키워드 기사 50개 출력
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from pandas import DataFrame
import os

# 사용자로부터 검색 키워드 입력 받기
query = input('검색 키워드를 입력하세요 : ')


# 네이버 뉴스 검색 URL
news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

# Selenium WebDriver 설정
driver = webdriver.Chrome()
driver.get(news_url.format(query))

SCROLL_PAUSE_TIME = 2
news_dict = {}
idx = 0
last_height = driver.execute_script("return document.body.scrollHeight")

print('크롤링 중...')
news_num = 50
while idx < news_num:
    # 페이지의 HTML 소스를 가져와 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    items = soup.select('.news_tit')
    
    for item in items:
        if idx >= news_num:
            break
        title = item.get_text()
        url = item.get('href')
        if title not in news_dict.values():
            news_dict[idx] = {'title': title, 'url': url}
            idx += 1
        
    
    # 페이지를 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 새로운 높이가 로드될 때까지 대기
    time.sleep(SCROLL_PAUSE_TIME)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # 더 이상 스크롤할 내용이 없으면 종료
    last_height = new_height

driver.quit()

print('크롤링 완료')

# 데이터프레임 변환
print('데이터프레임 변환')
news_df = DataFrame(news_dict).T

# 결과 출력
print(news_df)


크롤링 중...
크롤링 완료
데이터프레임 변환
                                       title  \
0       [단독] LG전자 'GPU 팜' 구축…“AI기술로 고객경험 혁신”   
1   영상 예술로 파고든 인공지능...국내 최초 'AI·메타버스 영화제' 열려   
2              '연일 무더위' 삼성·LG, 똑똑한 AI에어컨 맞대결   
3       한·일·중 교육장관, 4년5개월만 서울서 회담…AI교육·유학 논의   
4           SKT, 통신·기술 연합체 TM포럼서 AI 동맹 확장 나서   
5          'AI 돌풍' 반도체 호황 지속…5월 ICT 75억달러 흑자   
6           SKT 유영상, 글로벌 통신·기술 연합체 행사서 AI 연설   
7       "AI시대, 소통·창의력 지닌 리더 필요…인문학 더 중요해질 것"   
8        '원더랜드' 공유 목소리가 AI? 김태용 감독도 똑같아서 놀랐다   
9           "AI 열풍에 전기수요 폭증"…변압기업체 '거침없는 질주'   
10      [단독] LG전자 'GPU 팜' 구축…“AI기술로 고객경험 혁신”   
11  영상 예술로 파고든 인공지능...국내 최초 'AI·메타버스 영화제' 열려   
12             '연일 무더위' 삼성·LG, 똑똑한 AI에어컨 맞대결   
13      한·일·중 교육장관, 4년5개월만 서울서 회담…AI교육·유학 논의   
14          SKT, 통신·기술 연합체 TM포럼서 AI 동맹 확장 나서   
15         'AI 돌풍' 반도체 호황 지속…5월 ICT 75억달러 흑자   
16          SKT 유영상, 글로벌 통신·기술 연합체 행사서 AI 연설   
17      "AI시대, 소통·창의력 지닌 리더 필요…인문학 더 중요해질 것"   
18       '원더랜드' 공유 목소리가 AI? 김태용 감독도 똑같아서 놀랐다   
19          "A

In [56]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   60 non-null     object
 1   url     60 non-null     object
dtypes: object(2)
memory usage: 1.4+ KB


Q. 검색 키워드와 뉴스 기사 수를 입력한 후 다음 사항을 수행하는 크롤링 프로그램을 작성하세요.
- 크롤링한 내용에서 타이틀과 링크를 추출 
- 추출한 데이터를 판다스 데이터 프레임으로 변환하고 csv 파일로 저장
- csv 파일을 다시 불러와서 데이터 프레임으로 출력 

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from pandas import DataFrame
import os

# 사용자로부터 검색 키워드와 필요한 뉴스 기사 수 입력 받기
query = input('검색 키워드를 입력하세요 : ')
news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))

# 네이버 뉴스 검색 URL
news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

# Selenium WebDriver 설정
driver = webdriver.Chrome()
driver.get(news_url.format(query))

SCROLL_PAUSE_TIME = 2
news_dict = {}
idx = 0
last_height = driver.execute_script("return document.body.scrollHeight")

print('크롤링 중...')

while idx < news_num:
    # 페이지의 HTML 소스를 가져와 BeautifulSoup으로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    items = soup.select('.news_tit')
    
    for item in items:
        if idx >= news_num:
            break
        title = item.get_text()
        url = item.get('href')
        news_dict[idx] = {'title': title, 'url': url}
        idx += 1
    
    if idx >= news_num:
        break

    # 페이지를 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 새로운 높이가 로드될 때까지 대기
    time.sleep(SCROLL_PAUSE_TIME)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # 더 이상 스크롤할 내용이 없으면 종료
    last_height = new_height

driver.quit()

print('크롤링 완료')

# 데이터프레임 변환
print('데이터프레임 변환')
news_df = DataFrame(news_dict).T

# 결과 출력
# print(news_df)


folder_path = os.getcwd()
file_name = f'네이버뉴스_{query}.csv'
news_df.to_csv(file_name,index=False)
print(f'파일 저장 완료 | 경로 : {folder_path}\\{file_name}')
df = pd.read_csv(file_name)
df

크롤링 중...
크롤링 완료
데이터프레임 변환
파일 저장 완료 | 경로 : e:\workspace\kdt\web\crawling\네이버뉴스_ai.csv


,title,url
0,[단독] LG전자 'GPU 팜' 구축…“AI기술로 고객경험 혁신”,https://www.sedaily.com/NewsView/2DAHKXLLWC
1,"'연일 무더위' 삼성·LG, 똑똑한 AI에어컨 맞대결",https://www.bloter.net/news/articleView.html?i...
2,"한·일·중 교육장관, 4년5개월만 서울서 회담…AI교육·유학 논의",https://www.newsis.com/view/NISX20240614_00027...
3,영상 예술로 파고든 인공지능...국내 최초 'AI·메타버스 영화제' 열려,https://www.ytn.co.kr/_ln/0115_202406161624230667
4,"SKT, 통신·기술 연합체 TM포럼서 AI 동맹 확장 나서",https://www.yna.co.kr/view/AKR2024061601590001...
5,'AI 돌풍' 반도체 호황 지속…5월 ICT 75억달러 흑자,https://www.news1.kr/articles/5448155
6,"SKT 유영상, 글로벌 통신·기술 연합체 행사서 AI 연설",https://isplus.com/article/view/isp202406160098
7,"""AI시대, 소통·창의력 지닌 리더 필요…인문학 더 중요해질 것""",https://www.hankyung.com/article/2024061694841
8,'원더랜드' 공유 목소리가 AI? 김태용 감독도 똑같아서 놀랐다,https://www.joongang.co.kr/article/25256618
9,"""AI 열풍에 전기수요 폭증""…변압기업체 '거침없는 질주'",https://www.hankyung.com/article/2024061695161
